# Parsing votes
> Downloading & parsing votes Aafter downloading xlsx files behind the links on `https://www.bundestag.de/parlament/plenum/abstimmung/liste`.

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup

In [ ]:
from fastcore.all import *

import bundestag.logging as logging
import bundestag.paths as paths
from bundestag import html_parsing as hp

logger = logging.logger
logger.setLevel("DEBUG")

_paths = paths.get_paths("../data")
_paths

In [ ]:
dry = True

In [ ]:
_paths.make_raw_paths(dry=False)

## Collecting URIs for `.xlsx`/`.xls` documents from `.htm` files

`.xlsx` / `.xls` will be referred as "sheet" files.

In [ ]:
# path = Path("../bundestag.de_data")
# html_path = path / "htm_files"
# sheet_path = path / "sheets"

In [ ]:
html_path = _paths.raw_bundestag_html

In [ ]:
html_file_paths = hp.get_file_paths(html_path, pattern=hp.RE_HTM)
html_file_paths[:3]

In [ ]:
hp.test_file_paths(html_file_paths, html_path)

In [ ]:
sheet_uris = hp.collect_sheet_uris(html_file_paths)
list(sheet_uris.items())[:3], list(sheet_uris.items())[-3:]

In [ ]:
hp.test_sheet_uris(sheet_uris)

## Downloading sheet files

In [ ]:
sheet_path = _paths.raw_bundestag_sheets

In [ ]:
uri = sheet_uris[
    "10.09.2020: Abstrakte Normenkontrolle - Düngeverordnung (Beschlussempfehlung)"
]
hp.download_sheet(uri, sheet_path=sheet_path, dry=dry)

In [ ]:
hp.download_multiple_sheets(sheet_uris, sheet_path=sheet_path, nmax=3, dry=dry)

In [ ]:
file_title_maps = hp.get_file2poll_maps(sheet_uris, sheet_path)
hp.test_file_title_maps(file_title_maps, sheet_uris)

## Loading sheets into DataFrames

Collecting the `xlsx` and `xls` file names

In [ ]:
sheet_files = hp.get_file_paths(sheet_path, pattern=hp.RE_FNAME)
sheet_files

Reading files into dataframes

In [ ]:
sheet_file = sheet_files[0]
df = hp.get_sheet_df(sheet_file, file_title_maps=file_title_maps)
hp.test_get_sheet_df(df)
df.head().T

Squishing vote columns

In [ ]:
df_squished = hp.get_squished_dataframe(df)
hp.test_squished_df(df_squished, df)
df_squished.head(3).T

Setting some dtypes

In [ ]:
df_squished = hp.set_sheet_dtypes(df_squished)

Loading multiple sheets into dataframes

In [ ]:
df = hp.get_multiple_sheets_df(sheet_files, file_title_maps=file_title_maps)
df.head().T

Doing all the above

In [ ]:
df = hp.get_multiple_sheets(html_path, sheet_path, nmax=3, dry=dry)
df.head().T

Writing to disk

In [ ]:
df_path = _paths.preprocessed_bundestag / "bundestag.de_votes.parquet"
logger.info(f"Writing to {df_path}")

In [ ]:
_paths.make_preprocessed_paths(dry=False)

In [ ]:
df.to_parquet(df_path)